In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import random
import itertools
from sklearn.metrics import mean_squared_error
import theano
from keras import optimizers
from tensorflow.keras import datasets, layers, models
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from tqdm import tqdm
data = pd.read_excel('PCC6803 Promoter and reads 100bp.xlsx')
#model=tf.keras.models.load_model('100bp promoter with Random Search [100 iteration]).h5')

Using TensorFlow backend.


In [22]:
sorted_data= []

for i in range(3712):
    if data['Promoter'][i][-7] == 'T' and data['Promoter'][i][-9] == 'A' and data['Promoter'][i][-12:-10] == 'TA':
        sorted_data.append([data['Promoter'][i],data['Reads'][i]])
for j in range(3712):
    if data['Promoter'][j][-5] == 'T' and data['Promoter'][i][-7] == 'A' and data['Promoter'][j][-10:-8] == 'TA':
        sorted_data.append([data['Promoter'][j],data['Reads'][j]])
for k in range(3712):        
    if data['Promoter'][k][-6] == 'T' and data['Promoter'][i][-8] == 'A' and data['Promoter'][k][-11:-9] == 'TA':
        sorted_data.append([data['Promoter'][k],data['Reads'][k]])

In [24]:
def one_hot_encoding(seqs):
    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4)))
    n = len(seqs)
    total_width = 120
    X = np.zeros((n,1,4,total_width))
    for i in range(n):
        seq = seqs[i][0]
        for b in range(len(seq)):
            X[i,0,base_dict[seq[b]], b+10+100-len(seq)] = 1
    lio = []
    for i in range(n):
        lio.append(seqs[i][1])
    X = X.astype(theano.config.floatX)
    y = np.asarray(lio, dtype = theano.config.floatX)[:,np.newaxis]
    return X, y, total_width

def one_hot_encoding_nat(df, seq_column, expression):
    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4)))
    n = len(df)
    total_width = df[seq_column].str.len().max()+20
    X = np.zeros((n,1,4,total_width))
    seqs = df[seq_column].values
    for i in range(n):
        seq = seqs[i]
        for b in range(len(seq)):
            X[i,0,base_dict[seq[b]], b+10+100-len(seq)] = 1    
    X = X.astype(theano.config.floatX)
    y = np.asarray(df[expression].values, dtype = theano.config.floatX)[:,np.newaxis]
    return X, y, total_width

X, y, _ = one_hot_encoding(sorted_data)
Xtot, ytot, _ = one_hot_encoding_nat(data,'Promoter','Reads')
#y= np.log2(y+1)

In [25]:
y= np.log2(y+1)
ytot= np.log2(ytot+1)

In [26]:
hyperdict = {'con1_num' : [4,8,16,32],'con1_len' : [6,12,18,24], 'con2_prob' : [0],
             'con2_num' : [4,8,16,32],'con2_len' : [6,12,18,24],'den_prob' : [0,1],
             'den1_len' : [4,8,16,32], 'droprate' : [0.1,0.2,0.3,0.5] , 'batch_size' :[32,64], 'epochs' : [100]}

In [27]:
# Random search를 위한 함수 create_model 제작

def create_model(con1n = None, con1l= None,con2p = None, con2n= None, con2l= None,denp = None,den1= None, dout = None):
    model= models.Sequential()
    model.add(layers.Conv2D(con1n,(4, con1l), activation = 'relu', data_format = 'channels_first', input_shape = (1,4,120)))
    model.add(layers.Dropout(dout))
    #model.add(layers.MaxPooling2D((1,plen), data_format = 'channels_first'))
    if con2p == 1:     
        model.add(layers.Conv2D(con2n,(1,con2l),activation = 'relu' ,data_format = 'channels_first'))
        model.add(layers.Dropout(dout))
    model.add(layers.Flatten())
    model.add(layers.Dropout(dout))
    if denp == 1:
        model.add(layers.Dense(den1, activation = 'relu'))
    model.add(layers.Dense(1))
    return model

In [28]:
import time
import kFold
start = time.time() 
instant_val = 0
total_val = 10000
trial = 100
kfold = kFold(n_split = 5)
oplist = []
for X, y in kfold.split(X):
    c1n = random.choice(hyperdict['con1_num'])
    c1l = random.choice(hyperdict['con1_len'])
    c2p = random.choice(hyperdict['con2_prob'])
    c2n = random.choice(hyperdict['con2_num'])
    c2l = random.choice(hyperdict['con2_len'])
    denp =random.choice(hyperdict['den_prob'])
    den1 = random.choice(hyperdict['den1_len'])
    dout = random.choice(hyperdict['droprate'])
    ep = random.choice(hyperdict['epochs'])
    b_size = random.choice(hyperdict['batch_size'])
    model = create_model(c1n,c1l,c2p,c2n,c2l,denp,den1,dout)
    model.compile(optimizer = 'Adam', loss = 'mean_squared_error', metrics = ['mean_squared_error'])
    history = model.fit(X, y, epochs = ep, verbose = 0, batch_size = b_size, validation_split = 0.2)    
    instant_val = history.loss[-1]
    if total_val >instant_val:
        model.save('Model with refined data.h5')
print("time :", time.time() - start)

0 []
time : 25.202193021774292
0.1857274674323674
